In [ ]:

import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


./test.csv
./train.csv
./.config/.last_opt_in_prompt.yaml
./.config/gce
./.config/config_sentinel
./.config/.last_update_check.json
./.config/active_config
./.config/.last_survey_prompt.yaml
./.config/logs/2023.05.01/13.35.21.275622.log
./.config/logs/2023.05.01/13.36.23.672826.log
./.config/logs/2023.05.01/13.35.55.726719.log
./.config/logs/2023.05.01/13.35.47.453195.log
./.config/logs/2023.05.01/13.36.22.881789.log
./.config/logs/2023.05.01/13.34.56.313186.log
./.config/configurations/config_default
./sample_data/anscombe.json
./sample_data/README.md
./sample_data/mnist_train_small.csv
./sample_data/california_housing_test.csv
./sample_data/california_housing_train.csv
./sample_data/mnist_test.csv


In [ ]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')
print(train_data.shape)
print(train_data.columns)
print(test_data.shape)
train_data.head()

(7613, 5)
Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')
(3263, 4)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_data = train_data.drop('id',axis = 1)
test_data = test_data.drop('id',axis = 1)

In [ ]:
test_data = test_data.fillna('')
train_data = train_data.fillna('')

In [ ]:
import re
def keyword_correction(x):
    try:
        x = x.split("%20")
        x = ' '.join(x)
        return x
    except:
        return x

In [ ]:
train_data['keyword'] = train_data['keyword'].apply(lambda x: keyword_correction(x))
test_data['keyword'] = test_data['keyword'].apply(lambda x: keyword_correction(x))

In [ ]:
from nltk.corpus import stopwords
import string
from bs4 import BeautifulSoup
def text_cleaning(text):
    forbidden_words = set(stopwords.words('english'))
    if text:
        text = ' '.join(text.split('.'))
        text = re.sub(r'\s+', ' ', re.sub('[^A-Za-z0-9]', ' ', text.strip().lower())).strip()
        text = re.sub(r'\W+', ' ', text.strip().lower()).strip()
        text = [word for word in text.split() if word not in forbidden_words]
        return text
    return []

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"couldnt" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"doesnt" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"havent" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" w: "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"shouldnt" : "should not",
"that's" : "that is",
"thats" : "that is",
"there's" : "there is",
"theres" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"theyre":  "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"}

puncts = puncts + list(string.punctuation)

def clean_text(x):
    x = str(x).replace("\n","")
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('\d+', ' ', x)
    return x


def replace_typical_misspell(text):
    mispellings_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))

    def replace(match):
        return mispell_dict[match.group(0)]

    return mispellings_re.sub(replace, text)

def remove_space(string):
    string = BeautifulSoup(string).text.strip().lower()
    string = re.sub(r'((http)\S+)', 'http', string)
    string = re.sub(r'\s+', ' ', string)
    return string


def clean_data(df, columns: list):
    
    for col in columns:
        df[col] = df[col].apply(lambda x: remove_space(x).lower())        
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))
        df[col] = df[col].apply(lambda x: clean_text(x))
        
    return df

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
for col in ['location','text']:
    train_data[col] = train_data[col].apply(lambda x: ' '.join(text_cleaning(x)))
    test_data[col] = test_data[col].apply(lambda x: ' '.join(text_cleaning(x)))
train_data = clean_data(train_data,['keyword','text'])
test_data = clean_data(test_data,['keyword','text'])

In [ ]:
!python -m spacy download en_core_web_lg

2023-05-03 14:22:47.637053: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.5.0/en_core_web_lg-3.5.0-py3-none-any.whl (587.7 MB)


In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
def location_detection(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append(ent)
    if len(entities)>0:
        return 1
    return 0

In [ ]:
train_data['original_locations'] = train_data['location'].apply(lambda x: location_detection(x))
test_data['original_locations'] = test_data['location'].apply(lambda x: location_detection(x))

In [ ]:
spam_locations = ['place','room','home','somewhere','nowhere','everywhere','location',
                  'dope','kidding','moon','wherever','dimension','world','fvck','fuck','beside']
def is_location_spammy(text):
    for word in spam_locations:
        if word in text:
            return 1
    return 0

In [ ]:
train_data['Is_location_spam'] = train_data['location'].apply(lambda x: is_location_spammy(x))
test_data['Is_location_spam'] = test_data['location'].apply(lambda x: is_location_spammy(x))


In [ ]:
def digit_counter(text):
    """detects any digit in any token and counts
       once par token."""
    sum_number = 0
    doc = nlp(text)
    for token in doc:
        sum_number += bool(re.search(r'\d', token.text))*1
    return sum_number    

In [ ]:
train_data['digit_count_location'] = train_data['location'].apply(lambda x: digit_counter(x))
test_data['digit_count_location'] = test_data['location'].apply(lambda x: digit_counter(x))

In [ ]:
disaster_tweets =' '.join(train_data[train_data['target'] == 1]['text'].tolist())
non_disaster_tweets = ' '.join(train_data[train_data['target'] == 0]['text'].tolist())

In [ ]:
import nltk
def return_top_words(text,words = 10):
    allWords = nltk.tokenize.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')
    allWordExceptStopDist = nltk.FreqDist(w.lower() for w in allWords if w not in stopwords)    
    mostCommontuples= allWordExceptStopDist.most_common(words)
    mostCommon = [tupl[0] for tupl in mostCommontuples]
    return mostCommon

In [ ]:
top_50_disaster_words = return_top_words(disaster_tweets,50)
top_50_nondisaster_words = return_top_words(non_disaster_tweets,50)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
top_200_disaster_words = return_top_words(disaster_tweets,400)
top_200_nondisaster_words = return_top_words(non_disaster_tweets,400)
top_disaster_exclusive = list(set(top_200_disaster_words).difference(set(top_200_nondisaster_words)))
top_nondisaster_exclusive = list(set(top_200_nondisaster_words).difference(set(top_200_disaster_words)))

In [ ]:
total_vocab = top_disaster_exclusive + top_nondisaster_exclusive

In [ ]:
for word in total_vocab:
    train_data['Is_'+word+'_present'] = train_data['text'].apply(lambda x: (word in x)*1)
    test_data['Is_'+word+'_present'] = test_data['text'].apply(lambda x: (word in x)*1)

<ipython-input-38-674d49083021>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data['Is_'+word+'_present'] = train_data['text'].apply(lambda x: (word in x)*1)
<ipython-input-38-674d49083021>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_data['Is_'+word+'_present'] = train_data['text'].apply(lambda x: (word in x)*1)
<ipython-input-38-674d49083021>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(ngram_range=(1, 3),
                         binary=True,
                         max_features = 5000,
                         smooth_idf=False)
X_train_tfidf = tf_idf.fit_transform(train_data['text'])
X_test_tfidf = tf_idf.transform(test_data['text'])
tf_kw = TfidfVectorizer(ngram_range = (1,2),
                        binary = True,
                        max_features = 1500,
                        smooth_idf = False)
kw_train_tfidf = tf_kw.fit_transform(train_data['keyword'])
kw_test_tfidf = tf_kw.transform(test_data['keyword'])
tf_location = TfidfVectorizer(ngram_range = (1,2),
                              binary = True,
                              max_features = 1500,
                              smooth_idf = False)
location_train_tfidf = tf_location.fit_transform(train_data['location'])
location_test_tfidf = tf_location.transform(test_data['location'])

In [ ]:
train_data = pd.concat([train_data,pd.DataFrame(X_train_tfidf.toarray(),
                                                columns = ['text_contains_'+ str(text) for text in tf_idf.get_feature_names_out()]),
                        pd.DataFrame(kw_train_tfidf.toarray(),
                                     columns = ['keyword_contains_'+str(text) for text in tf_kw.get_feature_names_out()]),
                        pd.DataFrame(location_train_tfidf.toarray(),
                                     columns = ['location_contains_'+str(text) for text in tf_location.get_feature_names_out()])],axis = 1)
test_data = pd.concat([test_data,pd.DataFrame(X_test_tfidf.toarray(),
                                              columns = ['text_contains_'+ str(text) for text in tf_idf.get_feature_names_out()]),
                       pd.DataFrame(kw_test_tfidf.toarray(),
                                    columns = ['keyword_contains_'+str(text) for text in tf_kw.get_feature_names_out()]),
                       pd.DataFrame(location_test_tfidf.toarray(),
                                    columns = ['location_contains_'+str(text) for text in tf_location.get_feature_names_out()])],axis = 1)

In [ ]:
def create_vec(dataframe):
    texts = dataframe['text'].tolist()
    vectors = []
    for doc in nlp.pipe(texts):
        vectors.append(list(doc.vector))
    df = pd.DataFrame(vectors,columns = ['vec_'+str(i) for i in range(300)])
    return df
vec_train = create_vec(train_data)
vec_test = create_vec(test_data)
train_data = pd.concat([train_data,vec_train],axis = 1)
test_data = pd.concat([test_data,vec_test],axis = 1)


In [ ]:
train_data = train_data.drop(['keyword','location','text'],axis = 1)
test_data = test_data.drop(['keyword','location','text'],axis = 1)

In [ ]:
X_train = train_data.drop('target',axis = 1)
Y_train = train_data['target']
print('target' in test_data.columns)

False


In [ ]:
len(train_data.columns)

7579

In [ ]:
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.metrics import classification_report
forest = rfc(n_estimators = 128,max_depth = 8,min_samples_split = 15,
             class_weight = {0:1,1:1.6},oob_score = True)
forest.fit(X_train,Y_train)
print(forest.oob_score_)
Y_pred_train = forest.predict(X_train)
print(classification_report(Y_pred_train,Y_train))

0.7140417706554578
              precision    recall  f1-score   support

           0       0.77      0.86      0.81      3855
           1       0.84      0.73      0.78      3758

    accuracy                           0.80      7613
   macro avg       0.80      0.80      0.80      7613
weighted avg       0.80      0.80      0.80      7613



In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

def get_auc_CV(model,X_train,Y_train):
    """
    Return the average AUC score from cross-validation.
    """
    # Set KFold to shuffle data before the split
    kf = StratifiedKFold(5, shuffle=True, random_state=1)

    # Get AUC scores
    auc = cross_val_score(
        model, X_train, Y_train, scoring="roc_auc", cv=kf)

    return auc.mean()

In [ ]:

import time
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
xgb = XGBClassifier(n_estimators=200,learning_rate = 0.2,max_depth = 8)
training_start = time.perf_counter()
xgb.fit(X_train, Y_train)
training_end = time.perf_counter()
prediction_start = time.perf_counter()
pred_final = xgb.predict(test_data)
pred_train = xgb.predict(X_train)
print(classification_report(Y_train,pred_train))
prediction_end = time.perf_counter()
#acc_xgb = (preds == y_test).sum().astype(float) / len(preds)*100
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start
#print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))
print("Time consumed for training: %4.3f" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      4342
           1       1.00      0.99      0.99      3271

    accuracy                           0.99      7613
   macro avg       0.99      0.99      0.99      7613
weighted avg       0.99      0.99      0.99      7613

Time consumed for training: 789.782
Time consumed for prediction: 3.66282 seconds


In [ ]:
import pickle

with open("xgb_model.pkl", "wb") as f:
    pickle.dump(xgb, f)